<a href="https://colab.research.google.com/github/achintiii/MachineLearningWork/blob/main/Traffic_Signs_Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics
import os
from ultralytics import YOLO
import random
import shutil
from glob import glob

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [2]:
model = YOLO('yolov8n.pt')

100%|██████████| 6.25M/6.25M [00:00<00:00, 68.6MB/s]


In [3]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive')

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [4]:
# Define the base path and subfolder paths
base_path = '/content/drive/My Drive/Traffic Signs Object Detection/Raw Data/ts'  # Update this path to your dataset folder
labels_subfolder = os.path.join(base_path, 'labels')
images_subfolder = os.path.join(base_path, 'images')

# Create the subfolders if they don't exist
os.makedirs(labels_subfolder, exist_ok=True)
os.makedirs(images_subfolder, exist_ok=True)

In [5]:
traffic_sign_yaml_content = """
path: /content/drive/My Drive/Traffic Signs Object Detection/Raw Data/ts

train: /content/drive/My Drive/Traffic Signs Object Detection/Raw Data/ts/images/train
val: /content/drive/My Drive/Traffic Signs Object Detection/Raw Data/ts/images/val

nc: 4  # number of classes
names: ['prohibitory', 'danger', 'mandatory', 'other']  # class names

augment:
  flipud: 0.5  # Randomly flip images vertically
  fliplr: 0.5  # Randomly flip images horizontally
  hsv_h: 0.015  # Image HSV-Hue augmentation (fraction)
  hsv_s: 0.7  # Image HSV-Saturation augmentation (fraction)
  hsv_v: 0.4  # Image HSV-Value augmentation (fraction)
  translate: 0.1  # Image translation (pixels)
  scale: 0.5  # Image scale (fraction)
  shear: 0.0  # Image shear (degrees)
  perspective: 0.0  # Image perspective (fraction)
  rotate: 0.0  # Image rotation (degrees)
  mosaic: 1.0  # Mosaic augmentation probability
  mixup: 0.0  # Mixup augmentation probability
"""
traffic_sign_yaml_path = '/content/drive/My Drive/Traffic Signs Object Detection/Raw Data/ts/dataset2.yaml'
with open(traffic_sign_yaml_path, 'w') as f:
    f.write(traffic_sign_yaml_content)


In [6]:
# Check if directories exist
print("Checking if directories exist...")
!ls '/content/drive/My Drive/Traffic Signs Object Detection/Raw Data/ts/images/train'
!ls '/content/drive/My Drive/Traffic Signs Object Detection/Raw Data/ts/images/val'
!ls '/content/drive/My Drive/Traffic Signs Object Detection/Raw Data/ts/labels/train'
!ls '/content/drive/My Drive/Traffic Signs Object Detection/Raw Data/ts/labels/val'


Checking if directories exist...
00001.jpg  00079.jpg  00169.jpg  00253.jpg  00349.jpg  00444.jpg  00542.jpg  00679.jpg	00791.jpg
00002.jpg  00080.jpg  00170.jpg  00254.jpg  00352.jpg  00445.jpg  00544.jpg  00681.jpg	00794.jpg
00003.jpg  00082.jpg  00171.jpg  00255.jpg  00353.jpg  00446.jpg  00551.jpg  00682.jpg	00795.jpg
00004.jpg  00083.jpg  00172.jpg  00256.jpg  00355.jpg  00447.jpg  00552.jpg  00685.jpg	00797.jpg
00005.jpg  00084.jpg  00173.jpg  00257.jpg  00356.jpg  00448.jpg  00554.jpg  00688.jpg	00801.jpg
00006.jpg  00085.jpg  00174.jpg  00258.jpg  00357.jpg  00451.jpg  00556.jpg  00689.jpg	00802.jpg
00007.jpg  00086.jpg  00175.jpg  00259.jpg  00358.jpg  00452.jpg  00557.jpg  00690.jpg	00805.jpg
00008.jpg  00088.jpg  00176.jpg  00260.jpg  00359.jpg  00453.jpg  00560.jpg  00691.jpg	00806.jpg
00009.jpg  00089.jpg  00178.jpg  00261.jpg  00360.jpg  00455.jpg  00566.jpg  00692.jpg	00807.jpg
00010.jpg  00090.jpg  00179.jpg  00263.jpg  00362.jpg  00456.jpg  00567.jpg  00694.jpg	00808.j

In [9]:
# Function to clear and reload files
def reload_files(label_dir):
    label_files = glob(os.path.join(label_dir, '*.txt'))
    for label_file in label_files:
        with open(label_file, 'r') as f:
            lines = f.readlines()
            print(f"Contents of {label_file}:")
            for line in lines:
                print(line.strip())

# Reload files in train and val directories
print("Reloading files in train directory:")
reload_files('/content/drive/My Drive/Traffic Signs Object Detection/Raw Data/ts/labels/train')
print("Reloading files in val directory:")
reload_files('/content/drive/My Drive/Traffic Signs Object Detection/Raw Data/ts/labels/val')

Reloading files in train directory:
Contents of /content/drive/My Drive/Traffic Signs Object Detection/Raw Data/ts/labels/train/00004.txt:
1 0.6856617647058824 0.469375 0.05073529411764706 0.08375
0 0.6841911764705882 0.54125 0.03602941176470588 0.065
Contents of /content/drive/My Drive/Traffic Signs Object Detection/Raw Data/ts/labels/train/00003.txt:
0 0.5540441176470589 0.568125 0.016911764705882352 0.02875
0 0.5536764705882353 0.596875 0.016176470588235296 0.02875
1 0.5536764705882353 0.534375 0.023529411764705882 0.03875
Contents of /content/drive/My Drive/Traffic Signs Object Detection/Raw Data/ts/labels/train/00005.txt:
0 0.9029411764705882 0.27625 0.08235294117647059 0.1425
Contents of /content/drive/My Drive/Traffic Signs Object Detection/Raw Data/ts/labels/train/00001.txt:
0 0.7378676470588236 0.5125 0.030147058823529412 0.055
0 0.3044117647058823 0.65375 0.041176470588235294 0.0725
1 0.736764705882353 0.453125 0.04264705882352941 0.06875
Contents of /content/drive/My Drive/T

In [11]:
model.train(data=traffic_sign_yaml_path,epochs = 25)

Ultralytics YOLOv8.2.74 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
